In [ ]:
# Importing the Keras libraries and packages
from keras.layers import Input, Flatten, Dense
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.preprocessing import image
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Input, Flatten, Dense
from keras.callbacks import Callback, ModelCheckpoint
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
import warnings 
warnings.filterwarnings('ignore')
img_width, img_height = 224, 224

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Input parameter
train_data_dir ='/content/drive/My Drive/Dataset2/sign_data.zip (Unzipped Files)/sign_data/train'
val_data_dir ='/content/drive/My Drive/Dataset2/sign_data.zip (Unzipped Files)/sign_data/test'
TEST_DIR='/content/drive/My Drive/Dataset2/sign_data.zip (Unzipped Files)/sign_data/TEST1'
filepath = '/content/drive/My Drive/Dataset2/sign_data.zip (Unzipped Files)/sign_data/vgg16_GDPS_xfer_weights.hdf5'
nb_epochs = 50
TEST_DIR='/content/drive/My Drive/Dataset2/sign_data.zip (Unzipped Files)/sign_data/TEST1/'

SIGNATURE_CLASSES = ['001', '002', '003','004','006','009','012','013','014','015','016','017','018','019','020','021','022','023','024','025','026','027','028','029','030','031','032','033','034','035','036','037','038','039','040','041','042','043','044','045','046','047','048','049','050','051','052','053','054','055','056','057','058','059','060','061','062','063','064','065','066','067','068','069']
print("Input parameters are assigned")

Input parameters are assigned


In [ ]:
# image data generation
train_data_dir ='/content/drive/My Drive/Dataset2/sign_data.zip (Unzipped Files)/sign_data/train/'
val_data_dir ='/content/drive/My Drive/Dataset2/sign_data.zip (Unzipped Files)/sign_data/test/'
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_data_dir, target_size=(img_width, img_height), 
                                                    batch_size=32,shuffle=True, class_mode='categorical')
validation_generator = test_datagen.flow_from_directory(val_data_dir, target_size=(img_width, img_height), 
                                                    batch_size=32,shuffle=True,class_mode='categorical')

Found 672 images belonging to 64 classes.
Found 224 images belonging to 64 classes.


In [ ]:
#Load the pretrained Network
vgg16_model = VGG16(include_top=False, weights='imagenet', input_tensor=None, input_shape=(img_height,img_width,3), pooling=None, classes=1000)
print("pretrained Network is loaded")

58900480/58889256 [==============================] - 2s 0us/step
pretrained Network is loaded


In [ ]:
vgg16_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [ ]:
# Freeze the layers
for layer in vgg16_model.layers:
    layer.trainable = False
print("Pretrained layers are freezed")

Pretrained layers are freezed


In [ ]:
model = Sequential()
model.add(vgg16_model)
#add fully connected layer:
input_layer=model.add(Flatten())
hidden_layer=model.add(Dense(128, activation='relu', name='hidden_layer'))
classification_layer=model.add(Dense(64, activation='relu', name='classification_layer'))
output_layer=model.add(Dense(64, activation='softmax', name='output_layer')) 
print("All layers top of pretrained layers are developed")

All layers top of pretrained layers are developed


In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print("Model is Complied")

Model is Complied


In [ ]:
checkpoint = ModelCheckpoint(filepath = '/content/drive/My Drive/Dataset2/sign_data.zip (Unzipped Files)/sign_data/train/weights-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [ ]:
history = model.fit_generator( train_generator, callbacks = callbacks_list, nb_epoch=nb_epochs, validation_data=validation_generator)
print('Training Completed!')

TypeError: ignored

In [ ]:
import os
test_files = [im for im in os.listdir(TEST_DIR)]
test = np.ndarray((len(test_files), img_width, img_height, 3), dtype=np.uint8)

for i, im in enumerate(test_files): 
    test[i] = read_image(TEST_DIR+im)
    
test_preds = model.predict(test, verbose=1)
submission = pd.DataFrame(test_preds, columns=SIGNATURE_CLASSES)
submission.insert(0, 'image', test_files)
submission.head()

submission.to_csv('/content/drive/My Drive/Dataset2/sign_data.zip (Unzipped Files)/sign_data/signatureResults.csv',index=False)

NameError: ignored

In [ ]:
# save model and architecture to single file
model.save('/content/drive/My Drive/Dataset2/sign_data.zip (Unzipped Files)/sign_data/signatureRecognition_VGG16folder_model.h5')
model.summary()

print("Saved model to disk")

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 25088)             0         
_________________________________________________________________
hidden_layer (Dense)         (None, 128)               3211392   
_________________________________________________________________
classification_layer (Dense) (None, 64)                8256      
_________________________________________________________________
output_layer (Dense)         (None, 64)                4160      
Total params: 17,938,496
Trainable params: 3,223,808
Non-trainable params: 14,714,688
_________________________________________________________________
Saved model to disk


In [ ]:
# Loading saved model from Drive.
from keras.models import load_model
model = load_model('/content/drive/My Drive/Dataset2/sign_data.zip (Unzipped Files)/sign_data/signatureRecognition_VGG16folder_model.h5')
print("Model is Loaded")
model.summary()

Model is Loaded
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 25088)             0         
_________________________________________________________________
hidden_layer (Dense)         (None, 128)               3211392   
_________________________________________________________________
classification_layer (Dense) (None, 64)                8256      
_________________________________________________________________
output_layer (Dense)         (None, 64)                4160      
Total params: 17,938,496
Trainable params: 3,223,808
Non-trainable params: 14,714,688
_________________________________________________________________


In [ ]:
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.models import Model
import numpy as np
from keras.models import load_model
import ipywidgets as widgets
from IPython.display import display
button = widgets.Button(description="Click to know Name " )
output = widgets.Output()

model = load_model('/content/drive/My Drive/Dataset2/sign_data.zip (Unzipped Files)/sign_data/signatureRecognition_VGG16_model.h5')
img_path = '/content/drive/My Drive/Dataset2/sign_data.zip (Unzipped Files)/sign_data/11_052.png'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

block4_pool_features = model.predict(x)
print(block4_pool_features)
label_index=block4_pool_features.argmax()
print(label_index)
def on_button_clicked(b):
  # Display the message within the output widget.
  with output:
    print("Signature is of  "  +SIGNATURE_CLASSES[label_index])

button.on_click(on_button_clicked)
display(button, output)

[[1.07391554e-10 2.33683515e-15 1.57052270e-16 3.63358993e-07
  2.06276402e-12 5.14657650e-10 5.26921388e-11 1.13121075e-10
  1.23407440e-09 1.19378029e-07 1.47737534e-11 7.59142138e-10
  1.12326326e-08 1.42528648e-07 1.41108870e-14 2.81835728e-08
  1.50759361e-09 1.66780173e-10 1.15318489e-05 1.17536537e-07
  1.68616657e-06 4.35483707e-06 1.82785120e-10 6.03514678e-08
  2.88961519e-06 2.22058446e-07 1.76756994e-11 3.87882001e-14
  1.33577229e-11 8.05355175e-12 1.53550554e-05 4.03257464e-17
  1.60578134e-06 3.29196522e-12 6.05244049e-11 7.52713913e-06
  2.23651156e-07 2.73915261e-16 1.03515124e-13 6.31263054e-12
  2.15222798e-10 5.15617194e-06 1.58936767e-10 6.50010482e-13
  1.65133188e-05 1.07744245e-06 9.99741256e-01 3.34271916e-10
  2.03903596e-06 1.76024058e-13 4.70585159e-13 1.54898061e-09
  1.12555400e-15 3.14180255e-08 1.81714175e-04 1.67795555e-09
  8.04142690e-12 1.23989317e-12 3.76780527e-08 7.23589721e-07
  1.44526769e-11 2.06033746e-09 3.21945102e-08 5.13824034e-06]]
46


Button(description='Click to know Name ', style=ButtonStyle())

Output()

In [ ]:
from keras.models import load_model

model = load_model('/content/drive/My Drive/Dataset2/sign_data.zip (Unzipped Files)/sign_data/signatureRecognition_VGG16folder_model.h5')
# Evaluate the model on the test data using `evaluate`
print('\n# Evaluate on test data')
results = model.evaluate(validation_generator)
print('test loss, test acc:', results)


# Evaluate on test data
7/7 [==============================] - 124s 19s/step - loss: 0.9546 - accuracy: 0.8973
test loss, test acc: [0.9545983672142029, 0.8973214030265808]


In [ ]:
# Extracting Features from classification Layer
from keras.models import Model
layer_name= 'classification_layer'
intermediate_layer_model = Model(inputs=model.input,
                                 outputs=model.get_layer(layer_name).output)
print("Imtermediate model is constructed")


Imtermediate model is constructed


In [ ]:
# Compilation of intermediate model
intermediate_layer_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print("Model is Complied")

Model is Complied


In [ ]:
# Saving intermediate model
intermediate_layer_model.save('/content/drive/My Drive/Dataset2/sign_data.zip (Unzipped Files)/sign_data/signatureRecognition_VGG16folder_Intermediatemodel.h5')
intermediate_layer_model.summary()

print("Saved Intermediate model to disk")

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16_input (InputLayer)     [(None, 224, 224, 3)]     0         
_________________________________________________________________
vgg16 (Functional)           (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 25088)             0         
_________________________________________________________________
hidden_layer (Dense)         (None, 128)               3211392   
_________________________________________________________________
classification_layer (Dense) (None, 64)                8256      
Total params: 17,934,336
Trainable params: 3,219,648
Non-trainable params: 14,714,688
_________________________________________________________________
Saved Intermediate model to disk


In [ ]:
# Loading Intermediate Model
from keras.models import load_model

model = load_model('/content/drive/My Drive/Dataset2/sign_data.zip (Unzipped Files)/sign_data/signatureRecognition_VGG16folder_Intermediatemodel.h5')
print("Intermediate model is loaded")

Intermediate model is loaded


In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16_input (InputLayer)     [(None, 224, 224, 3)]     0         
_________________________________________________________________
vgg16 (Functional)           (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 25088)             0         
_________________________________________________________________
hidden_layer (Dense)         (None, 128)               3211392   
_________________________________________________________________
classification_layer (Dense) (None, 64)                8256      
Total params: 17,934,336
Trainable params: 3,219,648
Non-trainable params: 14,714,688
_________________________________________________________________


In [ ]:
# Training Label feature identification(y_train)
import numpy as np
batch_size=32
sample_count=672
features = np.zeros(shape=(672, 64))  # Must be equal to the output of the convolutional base
labels = np.zeros(shape=(672, 64))
i = 0
for inputs_batch, labels_batch in train_generator:
  features_batch = model.predict(inputs_batch)
  features[i * batch_size: (i + 1) * batch_size] = features_batch
  labels[i * batch_size: (i + 1) * batch_size] = labels_batch
  i += 1
  if i*batch_size  >= sample_count:
    break
print(labels.shape)
print(features.shape)


(672, 64)
(672, 64)


In [ ]:
 print(labels)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]]


In [ ]:
#identification of training Labels
features_train=features
print(features_train.shape)
labels_train=labels
print(labels_train.shape)
print(labels)
#print(features_train)


(672, 64)
(672, 64)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]]


In [ ]:
# identification of test labels
import numpy as np
batch_size=32
sample_count=224
features_test = np.zeros(shape=(224, 64))  # Must be equal to the output of the convolutional base
labels_test = np.zeros(shape=(224,64))
i = 0
for inputs_batch, labels_batch in validation_generator:
  features_batch = model.predict(inputs_batch)
  features_test[i * batch_size: (i + 1) * batch_size] = features_batch
  labels_test[i * batch_size: (i + 1) * batch_size] = labels_batch
  i += 1
  if i*batch_size  >= sample_count:
    break
print(labels_test.shape)
print(features_test.shape)
print(labels_test)

(224, 64)
(224, 64)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:
# #identification of testing Labels
print(features_test.shape)
#labels_test=np.expand_dims(labels_test, axis=1)
print(labels_test.shape)
print(features_test)

(224, 64)
(224, 64)
[[ 0.          0.          5.60965729 ...  0.         16.59523201
   0.        ]
 [ 0.          8.49717045  4.9733057  ...  6.01159573 13.58762455
   0.        ]
 [ 0.          9.02406311  2.11862731 ...  2.5939858   6.42814064
   0.        ]
 ...
 [ 0.         10.40638733  7.64363956 ...  1.9954381   6.96088696
   0.        ]
 [ 0.         11.28313255 13.2598238  ...  7.30460787  8.41551876
   0.        ]
 [ 0.         16.24743271 14.20442486 ...  3.36550164  0.91419935
   0.        ]]


In [ ]:
print(labels_test)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:
# SVM
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(features_train)
print(X_train.shape)
print(labels_train.shape)
X_test = sc.transform(features_test)
from sklearn.utils import check_X_y
X, y = check_X_y(
            X_train, labels_train, multi_output=True)
# Fitting SVM to the Training set
from sklearn.svm import SVC
classifier = SVC(kernel = 'linear', random_state = 0)
classifier.fit(X,y)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(labels_test, y_pred)
print('Confusion matrix: ')
print(cm)
# calculate Accuracy
from sklearn.metrics import accuracy_score
print('Accuracy: %.3f' % (accuracy_score(labels_test, y_pred)*100))
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
# calculate precision
# Precision = TruePositives / (TruePositives + FalsePositives)
precision = precision_score(labels_test, y_pred, average='binary')
print('Precision: %.3f' % (precision*100))
# calculate recall
# Recall = TruePositives / (TruePositives + FalseNegatives)
recall = recall_score(labels_test, y_pred, average='binary')
print('Recall: %.3f' % (recall*100))
# F-Measure = (2 * Precision * Recall) / (Precision + Recall)
# calculate score
score = f1_score(labels_test, y_pred, average='binary')
print('F-Measure: %.3f' % (score*100))

(672, 64)
(672, 64)


ValueError: ignored

In [ ]:
# Kernel SVM
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(features_train)
X_test = sc.transform(features_test)

# Fitting KernelSVM to the Training set
from sklearn.svm import SVC
classifier = SVC(kernel = 'rbf', random_state = 0)
classifier.fit(X_train, labels_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(labels_test, y_pred)
print('Confusion matrix: ')
print(cm)
# calculate Accuracy
from sklearn.metrics import accuracy_score
print('Accuracy: %.3f' % (accuracy_score(labels_test, y_pred)*100))
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
# calculate precision
# Precision = TruePositives / (TruePositives + FalsePositives)
precision = precision_score(labels_test, y_pred, average='binary')
print('Precision: %.3f' % (precision*100))
# calculate recall
# Recall = TruePositives / (TruePositives + FalseNegatives)
recall = recall_score(labels_test, y_pred, average='binary')
print('Recall: %.3f' % (recall*100))
# F-Measure = (2 * Precision * Recall) / (Precision + Recall)
# calculate score
score = f1_score(labels_test, y_pred, average='binary')
print('F-Measure: %.3f' % (score*100))

ValueError: ignored

In [ ]:
# Random Forest
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(features_train)
X_test = sc.transform(features_test)

# Fitting KernelSVM to the Training set
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
classifier.fit(X_train, labels_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
#cm = confusion_matrix(labels_test, y_pred)
print('Confusion matrix: ')
#print(cm)
# calculate Accuracy
from sklearn.metrics import accuracy_score
print('Accuracy: %.3f' % (accuracy_score(labels_test, y_pred)*100))
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
# calculate precision
# Precision = TruePositives / (TruePositives + FalsePositives)
precision = precision_score(labels_test, y_pred, average='micro')
print('Precision: %.3f' % (precision*100))
# calculate recall
# Recall = TruePositives / (TruePositives + FalseNegatives)
recall = recall_score(labels_test, y_pred, average='micro')
print('Recall: %.3f' % (recall*100))
# F-Measure = (2 * Precision * Recall) / (Precision + Recall)
# calculate score
score = f1_score(labels_test, y_pred, average='micro')
print('F-Measure: %.3f' % (score*100))

Confusion matrix: 
Accuracy: 59.375
Precision: 97.794
Recall: 59.375
F-Measure: 73.889


In [ ]:
# Decision Tree
# Feature Scaling

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(features_train)
X_test = sc.transform(features_test)

# Fitting KernelSVM to the Training set
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
classifier.fit(X_train, labels_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
#cm = confusion_matrix(labels_test, y_pred)
print('Confusion matrix: ')
#print(cm)
# calculate Accuracy
from sklearn.metrics import accuracy_score
print('Accuracy: %.3f' % (accuracy_score(labels_test, y_pred)*100))
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
# calculate precision
# Precision = TruePositives / (TruePositives + FalsePositives)
precision = precision_score(labels_test, y_pred, average='micro')
print('Precision: %.3f' % (precision*100))
# calculate recall
# Recall = TruePositives / (TruePositives + FalseNegatives)
recall = recall_score(labels_test, y_pred, average='micro')
print('Recall: %.3f' % (recall*100))
# F-Measure = (2 * Precision * Recall) / (Precision + Recall)
# calculate score
score = f1_score(labels_test, y_pred, average='micro')
print('F-Measure: %.3f' % (score*100))

Confusion matrix: 
Accuracy: 70.982
Precision: 70.982
Recall: 70.982
F-Measure: 70.982


In [ ]:
# Naive Bayes
# Feature Scaling
from sklearn.preprocessing import StandardScaler
from sklearn.utils import check_X_y
sc = StandardScaler()
X_train = sc.fit_transform(features_train)
X_test = sc.transform(features_test)
X, y = check_X_y(X_train, labels_train, accept_sparse=False, accept_large_sparse=True, dtype='numeric', order=None, copy=False, force_all_finite=True, ensure_2d=True, allow_nd=False, multi_output=True, ensure_min_samples=1, ensure_min_features=1, y_numeric=True, estimator=None)
# Fitting KernelSVM to the Training set
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X, y)
# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
#cm = confusion_matrix(labels_test, y_pred)
print('Confusion matrix: ')
#print(cm)
# calculate Accuracy
from sklearn.metrics import accuracy_score
print('Accuracy: %.3f' % (accuracy_score(labels_test, y_pred)*100))
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
# calculate precision
# Precision = TruePositives / (TruePositives + FalsePositives)
precision = precision_score(labels_test, y_pred, average='micro')
print('Precision: %.3f' % (precision*100))
# calculate recall
# Recall = TruePositives / (TruePositives + FalseNegatives)
recall = recall_score(labels_test, y_pred, average='micro')
print('Recall: %.3f' % (recall*100))
# F-Measure = (2 * Precision * Recall) / (Precision + Recall)
# calculate score
score = f1_score(labels_test, y_pred, average='micro')
print('F-Measure: %.3f' % (score*100))

ValueError: ignored

In [ ]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(features_train)
X_test = sc.transform(features_test)

# Fitting KernelSVM to the Training set
from sklearn.neighbors import KNeighborsClassifier
classifier =  KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
classifier.fit(X_train, labels_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
#cm = confusion_matrix(labels_test, y_pred)
print('Confusion matrix: ')
#print(cm)
# calculate Accuracy
from sklearn.metrics import accuracy_score
print('Accuracy: %.3f' % (accuracy_score(labels_test, y_pred)*100))
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
# calculate precision
# Precision = TruePositives / (TruePositives + FalsePositives)
precision = precision_score(labels_test, y_pred, average='micro')
print('Precision: %.3f' % (precision*100))
# calculate recall
# Recall = TruePositives / (TruePositives + FalseNegatives)
recall = recall_score(labels_test, y_pred, average='micro')
print('Recall: %.3f' % (recall*100))
# F-Measure = (2 * Precision * Recall) / (Precision + Recall)
# calculate score
score = f1_score(labels_test, y_pred, average='micro')
print('F-Measure: %.3f' % (score*100))

Confusion matrix: 
Accuracy: 91.071
Precision: 92.308
Recall: 91.071
F-Measure: 91.685


In [ ]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression(random_state = 0)
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(features_train)
X_test = sc.transform(features_test)
from sklearn.utils import check_X_y
X, y = check_X_y(
            X_train, labels_train, multi_output=True)
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
classifier.fit(X, y)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(labels_test, y_pred)
print('Confusion matrix: ')
print(cm)
# calculate Accuracy
from sklearn.metrics import accuracy_score
print('Accuracy: %.3f' % (accuracy_score(labels_test, y_pred)*100))
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
# calculate precision
# Precision = TruePositives / (TruePositives + FalsePositives)
precision = precision_score(labels_test, y_pred, average='micro')
print('Precision: %.3f' % (precision*100))
# calculate recall
# Recall = TruePositives / (TruePositives + FalseNegatives)
recall = recall_score(labels_test, y_pred, average='micro')
print('Recall: %.3f' % (recall*100))
# F-Measure = (2 * Precision * Recall) / (Precision + Recall)
# calculate score
score = f1_score(labels_test, y_pred, average='micro')
print('F-Measure: %.3f' % (score*100))

ValueError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install streamlit

In [ ]:
%%writefile app.py
import streamlit as st 
from PIL import Image
import pickle
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from keras.preprocessing import image
import os
from werkzeug.utils import secure_filename
st.set_option('deprecation.showfileUploaderEncoding', False)
from keras.models import load_model
model = load_model('/content/drive/My Drive/Dataset2/sign_data.zip (Unzipped Files)/sign_data/signatureRecognition_VGG16folder_model.h5')
SIGNATURE_CLASSES = ['001', '002', '003','004','006','009','012','013','014','015','016','017','018','019','020','021','022','023','024','025','026','027','028','029','030','031','032','033','034','035','036','037','038','039','040','041','042','043','044','045','046','047','048','049','050','051','052','053','054','055','056','057','058','059','060','061','062','063','064','065','066','067','068','069']
html_temp = """
   <div class="" style="background-color:blue;" >
   <div class="clearfix">           
   <div class="col-md-12">
   <center><p style="font-size:40px;color:white;margin-top:10px;">Poornima Univeristy</p></center> 
   <center><p style="font-size:30px;color:white;margin-top:10px;">Deepak Moud, PHD Scholar(2018PUSCEPHDE07061)</p></center> 
   </div>
   </div>
   </div>
   """
st.markdown(html_temp,unsafe_allow_html=True)
  
st.title("""
        Offline Signature Features Identification using deep Convolutional Neural Network
         """
         )
file= st.file_uploader("Please upload signare for Recognition", type=("jpg", "png"))
import cv2
from  PIL import Image, ImageOps
def import_and_predict(image_data, model):
  #img = image.load_img(image_data, target_size=(224, 224))
  #image = image.img_to_array(img)
  #img_reshap= np.expand_dims(image, axis=0)
  #img_reshap = preprocess_input(img_reshap)
  size=(224, 224)
  image=ImageOps.fit(image_data, size, Image.ANTIALIAS)
  img=np.asarray(image)
  img_reshape=np.expand_dims(img, axis=1)
  img_reshape=img[np.newaxis,...]
  block4_pool_features = model.predict(img_reshape)
  label_index=block4_pool_features.argmax()
  print(block4_pool_features)
  result=SIGNATURE_CLASSES[label_index]
  return result
if file is None:
  st.header("Please upload an signature Image")
else:
  image=Image.open(file)
  #imagefile.save('uploads/' + secure_filename(imagefile.filename))
  # Save the file to ./uploads        
  #file_path = os.path.join('/content','uploads', secure_filename(f.filename))
  st.image(image, use_column_width=True)
  
if st.button("Predict-VGG16"):
  result=import_and_predict(image,model)
  st.success('VGG16 Model has predicted signature is of user  {}'.format(result))
if st.button("About"):
  st.text(" Deepak Moud")
  st.text("Under the Guidance of Dr. Rekha Jain")
html_temp = """
   <div class="" style="background-color:orange;" >
   <div class="clearfix">           
   <div class="col-md-12">
   <center><p style="font-size:20px;color:white;margin-top:10px;">Objective 3: Evalution of pre trained Model for signature recognition</p></center> 
   </div>
   </div>
   </div>
   """
st.markdown(html_temp,unsafe_allow_html=True)

Writing app.py


In [ ]:
!pip install pyngrok

     |████████████████████████████████| 746 kB 3.5 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.0.6-py3-none-any.whl size=19262 sha256=9d5437fe8f4cb985f66920b75ceff2213f2e3da38080b69040121f6d2618ddea
  Stored in directory: /root/.cache/pip/wheels/d5/8c/c4/8d9cbca4fa19bf64887b4a91914194bb9033f1a7cbb344d5ab
Successfully built pyngrok


In [ ]:
!ngrok authtoken 1oEm0wopEJyjrT38ULluwUKK5fq_7ai4ZocZJ2YuFuoiJfoMh

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
!nohup streamlit run  app.py &

nohup: appending output to 'nohup.out'


In [ ]:
from pyngrok import ngrok
url=ngrok.connect(port=8501)
url

<NgrokTunnel: "http://49bc-35-194-212-16.ngrok.io" -> "http://localhost:80">

In [ ]:
 !streamlit run --server.port 80 app.py


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:80
  External URL: http://35.194.212.16:80

2021-08-23 08:04:36.131996: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-23 08:04:36.140168: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-23 08:04:36.140749: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-23 08:04:36.141897: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning N

In [ ]:
!kill ngrok()